<a href="https://colab.research.google.com/github/julianikulski/director-experience/blob/main/analysis/regression_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression analysis
The datasets created by the `exploratory data analysis` notebook are used in this notebook to conduct fixed effects regression analysis.

In [1]:
# connecting to Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install the linearmodels package
!pip install linearmodels &> /dev/null


In [3]:
import pandas as pd
import numpy as np

import linearmodels as lm
from linearmodels.panel.data import PanelData
from sklearn.linear_model import HuberRegressor
import statsmodels.api as sm



In [4]:
# read in the datasets
env_df = pd.read_csv('/content/drive/My Drive/director-csr/env_score_dataset.csv')
soc_df = pd.read_csv('/content/drive/My Drive/director-csr/soc_score_dataset.csv')
env_df.drop(columns=['Unnamed: 0'], inplace=True)
soc_df.drop(columns=['Unnamed: 0'], inplace=True)
env_df.head()


,ticker,comp_name,isin,env_score,year_score,independent_dirs,board_size,roa,leverage,slack,board_tenure,control_year,supersector,environmental_exp,environmental_comm_exp,ceo_duality,banks,basic resources,chemicals,construction and materials,consumer products and services,energy,financial services,"food, beverage and tobacco",health care,industrial goods and services,insurance,media,"personal care, drug and grocery stores",real estate,retailers,technology,telecommunications,travel and leisure,utilities,2013,2014,2015,2016
0,mmm,3m co,us88579y1010,83.891263,2012,90.00,10.0,14.65,16.88,20.07,9.38,2011,industrial goods and services,0.400000,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,abt,abbott laboratories,us0028241000,52.133087,2012,88.24,13.0,9.20,25.27,23.09,8.91,2011,health care,0.461538,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,atvi,activision blizzard inc,us00507v1098,0.000000,2012,23.08,12.0,8.02,0.00,35.54,7.29,2011,consumer products and services,0.090909,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,adbe,adobe inc,us00724f1012,80.082004,2012,90.00,9.0,10.23,16.84,34.79,13.23,2011,technology,0.111111,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,aap,advance auto parts inc,us00751y1064,7.411255,2012,81.82,10.0,11.72,11.82,10.37,5.36,2011,retailers,0.181818,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [5]:
# get the names of the industries
industries = list(env_df['supersector'].unique())
industries


['industrial goods and services',
 'health care',
 'consumer products and services',
 'technology',
 'retailers',
 'utilities',
 'insurance',
 'real estate',
 'chemicals',
 'financial services',
 'food, beverage and tobacco',
 'travel and leisure',
 'personal care, drug and grocery stores',
 'automobiles and parts',
 'telecommunications',
 'basic resources',
 'energy',
 'banks',
 'media',
 'construction and materials']

In [6]:
def prep_data(df, dep_var, ind_var, effect):
    '''
    Function to split the data into X and y and create indices
    to prepare for estimation
    Args: df = dataframe
          dep_var = str; dependent variable name
          ind_var = str; independent variable name
          effect = str; name of effect
    Returns: X, y = dataframes
    '''
    if 'comm' in ind_var:
        var_to_be_removed = ind_var.replace('_comm', '')
    else:
        var_to_be_removed = ind_var.replace('exp', '')
        var_to_be_removed = var_to_be_removed+'comm_exp'

    if effect == 'industry':
        index_col = 'supersector'
        effect_to_be_removed = 'ticker'
    elif effect == 'entity':
        index_col = 'ticker'
        effect_to_be_removed = 'supersector'

    # define the variables
    df.rename(columns={'year_score':'time'}, inplace=True)
    new_df = df.set_index([index_col, 'time']).copy()
    y = new_df[[dep_var]].copy()
    # add all columns to be dropped together
    dummies = list(new_df.columns)[14:]
    dummies.extend(['comp_name', 'isin', dep_var,
                    var_to_be_removed, 'control_year'])
    dummies.append(effect_to_be_removed)
    X = new_df.drop(columns=dummies) 

    return X, y


In [7]:
# define X and y
X_soc_exp, y_soc_exp = prep_data(soc_df, 'soc_score', 'social_exp', 'industry')
# add a constant
X_soc_exp = sm.add_constant(X_soc_exp)
X_soc_comm, y_soc_comm = prep_data(soc_df, 'soc_score', 'social_comm_exp', 'industry')
# add a constant
X_soc_comm = sm.add_constant(X_soc_comm)
X_env_exp, y_env_exp = prep_data(env_df, 'env_score', 'environmental_exp', 'industry')
# add a constant
X_env_exp = sm.add_constant(X_env_exp)
X_env_comm, y_env_comm = prep_data(env_df, 'env_score', 'environmental_comm_exp', 'industry')
# add a constant
X_env_comm = sm.add_constant(X_env_comm)


In [8]:
# shapes of X and y
print(X_env_comm.shape)
print(y_env_comm.shape)


(2201, 9)
(2201, 1)


An explanation of how the covariance type `clustered` is calculated by linearmodels is given here: https://github.com/bashtage/linearmodels/blob/5c2b663314d6bd337eb601afb378927468fc9a5e/linearmodels/panel/covariance.py

In [9]:
# fixed effects regression 
# time and entity
mod = lm.PanelOLS(y_env_comm, X_env_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.1803
Estimator:,PanelOLS,R-squared (Between):,-0.1899
No. Observations:,2201,R-squared (Within):,0.1813
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1174
Time:,14:38:02,Log-likelihood,-1.009e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,59.648
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,23.906


In [10]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_comm.shape[0] - 1) / (X_env_comm.shape[0] - (X_env_comm.shape[1]-1) -1))
adjusted_r2


0.18192336269500148

In [11]:
# fixed effects regression 
# time and entity
mod = lm.PanelOLS(y_env_exp, X_env_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.1648
Estimator:,PanelOLS,R-squared (Between):,-0.3203
No. Observations:,2201,R-squared (Within):,0.1653
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0849
Time:,14:38:03,Log-likelihood,-1.011e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,53.485
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,21.372


In [12]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_exp.shape[0] - 1) / (X_env_exp.shape[0] - (X_env_exp.shape[1]-1) -1))
adjusted_r2


0.16595216537008417

In [13]:
# fixed effects regression 
# time and entity
mod = lm.PanelOLS(y_soc_exp, X_soc_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.1881
Estimator:,PanelOLS,R-squared (Between):,-0.1703
No. Observations:,2201,R-squared (Within):,0.1874
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1323
Time:,14:38:03,Log-likelihood,-9377.4
Cov. Estimator:,Clustered,,
,,F-statistic:,62.833
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,28.770


In [14]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_exp.shape[0] - 1) / (X_soc_exp.shape[0] - (X_soc_exp.shape[1]-1) -1))
adjusted_r2


0.18812297005401551

In [15]:
# fixed effects regression 
# time and entity
mod = lm.PanelOLS(y_soc_comm, X_soc_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.2070
Estimator:,PanelOLS,R-squared (Between):,-0.1166
No. Observations:,2201,R-squared (Within):,0.2066
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1582
Time:,14:38:03,Log-likelihood,-9351.4
Cov. Estimator:,Clustered,,
,,F-statistic:,70.792
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,35.838


In [16]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_comm.shape[0] - 1) / (X_soc_comm.shape[0] - (X_soc_comm.shape[1]-1) - 1))
adjusted_r2


0.2073961000641916

# Robustness check



## Removed outliers

Outliers were truncated at the 1st and 99th percentile

In [17]:
# read in the datasets without outliers
env_df = pd.read_csv('/content/drive/My Drive/director-csr/no_outliers_env.csv')
soc_df = pd.read_csv('/content/drive/My Drive/director-csr/no_outliers_soc.csv')
env_df.drop(columns=['Unnamed: 0'], inplace=True)
soc_df.drop(columns=['Unnamed: 0'], inplace=True)
env_df.head()


,env_score,independent_dirs,board_size,roa,leverage,slack,board_tenure,environmental_exp,environmental_comm_exp,ticker,comp_name,isin,year_score,control_year,supersector,ceo_duality
0,83.891263,90.00,10.0,14.65,16.88,20.07,9.38,0.400000,0.0,mmm,3m co,us88579y1010,2012,2011,industrial goods and services,0
1,52.133087,88.24,13.0,9.20,25.27,23.09,8.91,0.461538,0.0,abt,abbott laboratories,us0028241000,2012,2011,health care,0
2,80.082004,90.00,9.0,10.23,16.84,34.79,13.23,0.111111,0.0,adbe,adobe inc,us00724f1012,2012,2011,technology,0
3,7.411255,81.82,10.0,11.72,11.82,10.37,5.36,0.181818,0.0,aap,advance auto parts inc,us00751y1064,2012,2011,retailers,1
4,57.980280,87.50,13.0,4.85,34.79,21.86,7.41,0.200000,0.0,aep,NaN,us0255371017,2012,2011,utilities,0


In [18]:
# get the names of the industries
industries = list(env_df['supersector'].unique())
industries


['industrial goods and services',
 'health care',
 'technology',
 'retailers',
 'utilities',
 'insurance',
 'real estate',
 'chemicals',
 'financial services',
 'food, beverage and tobacco',
 'travel and leisure',
 'telecommunications',
 'basic resources',
 'energy',
 'banks',
 'automobiles and parts',
 'personal care, drug and grocery stores',
 'media',
 'consumer products and services',
 'construction and materials']

In [19]:
# define X and y
X_soc_exp, y_soc_exp = prep_data(soc_df, 'soc_score', 'social_exp', 'industry')
# add a constant
X_soc_exp = sm.add_constant(X_soc_exp)
X_soc_comm, y_soc_comm = prep_data(soc_df, 'soc_score', 'social_comm_exp', 'industry')
# add a constant
X_soc_comm = sm.add_constant(X_soc_comm)
X_env_exp, y_env_exp = prep_data(env_df, 'env_score', 'environmental_exp', 'industry')
# add a constant
X_env_exp = sm.add_constant(X_env_exp)
X_env_comm, y_env_comm = prep_data(env_df, 'env_score', 'environmental_comm_exp', 'industry')
# add a constant
X_env_comm = sm.add_constant(X_env_comm)


In [20]:
# fixed effects regression removed outliers
# time and entity
mod = lm.PanelOLS(y_env_comm, X_env_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.1382
Estimator:,PanelOLS,R-squared (Between):,-0.0539
No. Observations:,1397,R-squared (Within):,0.1354
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0813
Time:,14:38:04,Log-likelihood,-6201.5
Cov. Estimator:,Clustered,,
,,F-statistic:,27.363
Entities:,20,P-value,0.0000
Avg Obs:,69.850,Distribution:,"F(8,1365)"
Min Obs:,18.000,,
Max Obs:,198.00,F-statistic (robust):,12.909


In [21]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_comm.shape[0] - 1) / (X_env_comm.shape[0] - (X_env_comm.shape[1]-1) -1))
adjusted_r2


0.13614097545612383

In [22]:
# fixed effects regression removed outliers
# time and entity
mod = lm.PanelOLS(y_env_exp, X_env_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.1094
Estimator:,PanelOLS,R-squared (Between):,-0.1561
No. Observations:,1397,R-squared (Within):,0.1059
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0429
Time:,14:38:04,Log-likelihood,-6224.4
Cov. Estimator:,Clustered,,
,,F-statistic:,20.964
Entities:,20,P-value,0.0000
Avg Obs:,69.850,Distribution:,"F(8,1365)"
Min Obs:,18.000,,
Max Obs:,198.00,F-statistic (robust):,12.291


In [23]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_exp.shape[0] - 1) / (X_env_exp.shape[0] - (X_env_exp.shape[1]-1) -1))
adjusted_r2


0.1064997817464292

In [24]:
# fixed effects regression removed outliers
# time and entity
mod = lm.PanelOLS(y_soc_exp, X_soc_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.1419
Estimator:,PanelOLS,R-squared (Between):,-0.1062
No. Observations:,1700,R-squared (Within):,0.1397
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0978
Time:,14:38:04,Log-likelihood,-7146.4
Cov. Estimator:,Clustered,,
,,F-statistic:,34.472
Entities:,20,P-value,0.0000
Avg Obs:,85.000,Distribution:,"F(8,1668)"
Min Obs:,25.000,,
Max Obs:,230.00,F-statistic (robust):,22.669


In [25]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_exp.shape[0] - 1) / (X_soc_exp.shape[0] - (X_soc_exp.shape[1]-1) -1))
adjusted_r2


0.14040824730003063

In [26]:
# fixed effects regression removed outliers
# time and entity
mod = lm.PanelOLS(y_soc_comm, X_soc_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.1722
Estimator:,PanelOLS,R-squared (Between):,-0.0765
No. Observations:,1700,R-squared (Within):,0.1714
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1260
Time:,14:38:04,Log-likelihood,-7115.7
Cov. Estimator:,Clustered,,
,,F-statistic:,43.381
Entities:,20,P-value,0.0000
Avg Obs:,85.000,Distribution:,"F(8,1668)"
Min Obs:,25.000,,
Max Obs:,230.00,F-statistic (robust):,20.970


In [27]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_comm.shape[0] - 1) / (X_soc_comm.shape[0] - (X_soc_comm.shape[1]-1) - 1))
adjusted_r2


0.17221394072831087

## Winsorized outliers
Outliers were winsorized at the 1st and 99th percentile

In [28]:
# read in the datasets without outliers
env_df = pd.read_csv('/content/drive/My Drive/director-csr/winsorized_env.csv')
soc_df = pd.read_csv('/content/drive/My Drive/director-csr/winsorized_soc.csv')
env_df.drop(columns=['Unnamed: 0'], inplace=True)
soc_df.drop(columns=['Unnamed: 0'], inplace=True)
env_df.head()


,ticker,comp_name,isin,env_score,year_score,independent_dirs,board_size,roa,leverage,slack,board_tenure,control_year,supersector,environmental_exp,environmental_comm_exp,ceo_duality
0,mmm,3m co,us88579y1010,83.891263,2012,90.00,10.0,14.65,16.88,20.07,9.38,2011,industrial goods and services,0.400000,0.0,0
1,abt,abbott laboratories,us0028241000,52.133087,2012,88.24,13.0,9.20,25.27,23.09,8.91,2011,health care,0.461538,0.0,0
2,atvi,activision blizzard inc,us00507v1098,0.000000,2012,50.00,12.0,8.02,0.00,35.54,7.29,2011,consumer products and services,0.090909,0.0,1
3,adbe,adobe inc,us00724f1012,80.082004,2012,90.00,9.0,10.23,16.84,34.79,13.23,2011,technology,0.111111,0.0,0
4,aap,advance auto parts inc,us00751y1064,7.411255,2012,81.82,10.0,11.72,11.82,10.37,5.36,2011,retailers,0.181818,0.0,1


In [29]:
# get the names of the industries
industries = list(env_df['supersector'].unique())
industries


['industrial goods and services',
 'health care',
 'consumer products and services',
 'technology',
 'retailers',
 'utilities',
 'insurance',
 'real estate',
 'chemicals',
 'financial services',
 'food, beverage and tobacco',
 'travel and leisure',
 'personal care, drug and grocery stores',
 'automobiles and parts',
 'telecommunications',
 'basic resources',
 'energy',
 'banks',
 'media',
 'construction and materials']

In [30]:
# define X and y
X_soc_exp, y_soc_exp = prep_data(soc_df, 'soc_score', 'social_exp', 'industry')
# add a constant
X_soc_exp = sm.add_constant(X_soc_exp)
X_soc_comm, y_soc_comm = prep_data(soc_df, 'soc_score', 'social_comm_exp', 'industry')
# add a constant
X_soc_comm = sm.add_constant(X_soc_comm)
X_env_exp, y_env_exp = prep_data(env_df, 'env_score', 'environmental_exp', 'industry')
# add a constant
X_env_exp = sm.add_constant(X_env_exp)
X_env_comm, y_env_comm = prep_data(env_df, 'env_score', 'environmental_comm_exp', 'industry')
# add a constant
X_env_comm = sm.add_constant(X_env_comm)


In [31]:
# fixed effects regression winsorized outliers
# time and entity
mod = lm.PanelOLS(y_env_comm, X_env_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.2071
Estimator:,PanelOLS,R-squared (Between):,-0.2380
No. Observations:,2201,R-squared (Within):,0.2078
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1329
Time:,14:38:05,Log-likelihood,-1.005e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,70.835
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,41.168


In [32]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_comm.shape[0] - 1) / (X_env_comm.shape[0] - (X_env_comm.shape[1]-1) -1))
adjusted_r2


0.20859377786772415

In [33]:
# fixed effects regression winsorized outliers
# time and entity
mod = lm.PanelOLS(y_env_exp, X_env_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,env_score,R-squared:,0.1910
Estimator:,PanelOLS,R-squared (Between):,-0.3595
No. Observations:,2201,R-squared (Within):,0.1914
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1016
Time:,14:38:05,Log-likelihood,-1.007e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,63.999
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,39.163


In [34]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_env_exp.shape[0] - 1) / (X_env_exp.shape[0] - (X_env_exp.shape[1]-1) -1))
adjusted_r2


0.19210315922038892

In [35]:
# fixed effects regression winsorized outliers
# time and entity
mod = lm.PanelOLS(y_soc_exp, X_soc_exp, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.1972
Estimator:,PanelOLS,R-squared (Between):,-0.2033
No. Observations:,2201,R-squared (Within):,0.1964
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1373
Time:,14:38:05,Log-likelihood,-9353.6
Cov. Estimator:,Clustered,,
,,F-statistic:,66.581
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,30.065


In [36]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_exp.shape[0] - 1) / (X_soc_exp.shape[0] - (X_soc_exp.shape[1]-1) -1))
adjusted_r2


0.19712251981231627

In [37]:
# fixed effects regression winsorized outliers
# time and entity
mod = lm.PanelOLS(y_soc_comm, X_soc_comm, time_effects=True, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)
res


Dep. Variable:,soc_score,R-squared:,0.2164
Estimator:,PanelOLS,R-squared (Between):,-0.1532
No. Observations:,2201,R-squared (Within):,0.2158
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.1629
Time:,14:38:06,Log-likelihood,-9327.0
Cov. Estimator:,Clustered,,
,,F-statistic:,74.854
Entities:,20,P-value,0.0000
Avg Obs:,110.05,Distribution:,"F(8,2169)"
Min Obs:,35.000,,
Max Obs:,300.00,F-statistic (robust):,47.560


In [38]:
# calculate the adjusted 
r2 = res.rsquared_within

adjusted_r2 = (1 - (1 - r2))*((X_soc_comm.shape[0] - 1) / (X_soc_comm.shape[0] - (X_soc_comm.shape[1]-1) - 1))
adjusted_r2


0.2166070641204397